In [497]:
import random

In [88]:
def gen_chromosome(bits):
    return [random.randrange(0, 2) for row in range(bits)]

In [98]:
def get_individual(bits, chromoQuantity):
    return [gen_chromosome(bits) for row in range(chromoQuantity)]

In [99]:
def gen_population(bits, chromoQuantity, popSize):
    return [get_individual(bits, chromoQuantity) for row in range(popSize)]

In [125]:
def popToDecimal(population):
    return [[toDecimal(chromo) for chromo in individual] for individual in population]

In [132]:
def popBoothFunc(decimalPop):
    return [boothFunc(individual[0], individual[1]) for individual in decimalPop]

## d. Implementacja selekcji najlepszych
- `population` - populacja osobników w reprezentacji binarnej, lista
- `percent` - procent najlepszych osobników, liczba całkowita
- `bestSelected` - najlepiej przystostowane osobniki z populacji, lista

In [320]:
def bestSelection(population, percent = 30):
    decimalPop = popToDecimal(population)
    popAdaptation = popBoothFunc(decimalPop)
    
    bestSelected = []
    N = int(len(popAdaptation) * percent / 100)
    for i in range(0, N):  
        best = max(popAdaptation)
        bestSelected.append(population[popAdaptation.index(best)])
        popAdaptation.remove(best) 
    return bestSelected

In [670]:
bestSelected = bestSelection(population[:], 5)
bestSelected

[[[1, 1, 1, 0, 0, 0, 1, 1, 0], [1, 1, 1, 1, 0, 0, 0, 1, 1]]]

## d. Implementacja selekcji turniejowej
- `population` - populacja osobników w reprezentacji binarnej, lista
- `k` -  wielkość turnieju, liczba całkowita
- `pop` - osobnik wybrany z populacji selekcją turniejową, lista

In [676]:
def tournamentSelection(population, k):
    pop = population[:]
    popSize = len(pop)
    
    if popSize == 1:
        return pop
    else:
        bestOfTournament = []
        # create tournaments using all individuals from population
        while(popSize > 1):
            popSize = len(pop)
            # select k random individuals from population to new tournament (without repetition)
            tournament = []
            if popSize >= k:
                for i in range(k):
                    randInd = random.choice(pop)
                    tournament.append(randInd)
                    pop.remove(randInd)
            elif popSize > 1:
                tournament = pop[:]
                pop.clear() 
            else:
                break
            # select the best individuals tournament (stochastic tournament selection)
            decimalTournament = popToDecimal(tournament)
            tournamentAdaptation = popBoothFunc(decimalTournament)
            bestAdaptation = random.choice(tournamentAdaptation)
            bestSelected = tournament[tournamentAdaptation.index(bestAdaptation)]
            bestOfTournament.append(bestSelected)
            
        return tournamentSelection(bestOfTournament, k)

In [677]:
bestOfTournament = tournamentSelection(population, k = 3)
bestOfTournament

[[[1, 0, 1, 0, 1, 1, 1, 1, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0]]]

## d. Implementacja selekcji kołem ruletki
- `population` - populacja osobników w reprezentacji binarnej, lista
- `bestSelected` - najlepiej przystostowany osobnik z populacji

Dodatkowo: 
1. implementacja funcji Booth dla selekcji kołem ruletki dla problemu minimalizacji
   - `decimalPop` - populacja osobników w reprezentacji dziesiętnej, lista
2. implementacja wyznaczenia dystrybucji osobników dla danej populacji
   - `population` - populacja osobników w reprezentacji binarnej, lista
   - `dist` - dystrybucje osobników z populacji, lista
3. implementacja prezentacji koła ruletki dla populacji
   - `population` - populacja osobników w reprezentacji binarnej, lista

In [447]:
def minProblemBoothFunc(decimalPop):
    return [1/boothFunc(individual[0], individual[1]) for individual in decimalPop]

In [448]:
def calculateDistribution(population):
    decimalPop = popToDecimal(population)
    popAdaptation = minProblemBoothFunc(decimalPop)
    
    # caculate population adaptations sum
    adaptationSum = sum(popAdaptation) 
    # calculate every individual probability
    prob = [adaptation/adaptationSum for adaptation in popAdaptation] 
    
    # calculate every individual probability
    dist = prob[:]
    for i in range(1, len(dist)):
        dist[i] += dist[i-1]
    return dist

In [449]:
def rouletteWheelSelection(population):
    populationDist = calculateDistribution(population)
        
    # spin roulette wheel and find the best individual
    rand = random.random()
    previous = 0
    bestIndex = -1
    for index, value in enumerate(populationDist):
        if(previous <= rand <= value):
            bestIndex = index
        previous = value
    return population[bestIndex]

In [451]:
def showRouletteWheel(population):
    populationDist = calculateDistribution(population)
    stringPop = []
    for individual in population:
        stringPop.append(str(individual))
    fig = go.Figure(data = [go.Pie(labels = stringPop, values = populationDist)])
    fig.show()

In [1144]:
import plotly.graph_objects as go
showRouletteWheel(population)

In [450]:
rouletteWheelSelection(population)

[[0, 0, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0, 0, 0, 1]]

# OBIEKTOWO

#### implementacja krzyżowania jedno, dwu i trzy punktowego oraz krzyżowania jednorodnego
- `x1` - chromosom pierwszy, lista
- `x2` - chromosom drugi, lista
- `crossoverProb` - prawdopodobieństwo krzyżowania, liczba z zakresu 0,1

In [756]:
def singlePointCrossover(x1, x2):
    point = random.randrange(1, len(x1))
    temp = x1[:point]
    x1[:point] = x2[:point]
    x2[:point] = temp
    return x1, x2

In [1108]:
def twoPointsCrossover(x1, x2):
    points = random.sample(range(1, len(x1)), 2)
    point1 = min(points)
    point2 = max(points)
    temp = x1[point1:point2]
    x1[point1:point2] = x2[point1:point2]
    x2[point1:point2] = temp
    return x1, x2

In [787]:
def threePointsCrossover(x1, x2):
    points = random.sample(range(1, len(x1)), 3)
    point1 = min(points)
    point2 = max(points)
    point3 = [i for i in points if i != point1 and i != point2][0]

    temp = x1[point1:point2]
    x1[point1:point2] = x2[point1:point2]
    x2[point1:point2] = temp
    temp = x1[point3:]
    x1[point3:] = x2[point3:]
    x2[point3:] = temp
    
    return x1, x2

In [1113]:
def probUniformCrossover(x1, x2, crossoverProb):
    for index in range(len(x1)):
        prob = random.random()
        if prob < crossoverProb:
            temp = x1[index]
            x1[index] = x2[index]
            x2[index] = temp
    return x1, x2

### Implementacja krzyżowania jedno, dwu i trzy punktowego oraz krzyżowania jednorodnego obiektowo
- `ind1` - pierwszy osobnik, obiekt typu Individual
- `ind2` - pierwszy osobnik, obiekt typu Individual
- `crossoverProb` - prawdopodobieństwo krzyżowania, liczba z zakresu 0,1

In [1105]:
def singlePointCrossoverObj(ind1, ind2):
    x1, x2 = singlePointCrossover(ind1.X.binary, ind2.X.binary)
    ind1.setX(x1)
    ind2.setX(x2)

    y1, y2 = singlePointCrossover(ind1.Y.binary, ind2.Y.binary)
    ind1.setY(y1)
    ind2.setY(y2)

In [1109]:
def twoPointsCrossoverObj(ind1, ind2):
    x1, x2 = twoPointsCrossover(ind1.X.binary, ind2.X.binary)
    ind1.setX(x1)
    ind2.setX(x2)

    y1, y2 = twoPointsCrossover(ind1.Y.binary, ind2.Y.binary)
    ind1.setY(y1)
    ind2.setY(y2)

In [1110]:
def threePointsCrossoverObj(ind1, ind2):
    x1, x2 = threePointsCrossover(ind1.X.binary, ind2.X.binary)
    ind1.setX(x1)
    ind2.setX(x2)

    y1, y2 = threePointsCrossover(ind1.Y.binary, ind2.Y.binary)
    ind1.setY(y1)
    ind2.setY(y2)

In [1141]:
def probUniformCrossoverObj(ind1, ind2, crossoverProb):
    x1, x2 = probUniformCrossover(ind1.X.binary, ind2.X.binary, crossoverProb)
    ind1.setX(x1)
    ind2.setX(x2)

    y1, y2 = probUniformCrossover(ind1.Y.binary, ind2.Y.binary, crossoverProb)
    ind1.setY(y1)
    ind2.setY(y2)

#### setBinary - implementacja binarnej reprezentacji chromosomu + konfiguracja dokładności
- `bits` - liczba bitów chromosomu, liczba całkowita

#### setDecimal - implementacja rozkodowania chromosomu - konwersji  z reprezentacji binarnej do reprezentacji dziesiętnej
- `binary` - chromosom w reprezentacji binarnej, tablica jednowymiarowa `ndarray`
- `decimal` - chromosom w reprezentacji dziesiętnej, liczba całkowita

#### onePointMutation, twoPointsMutation, edgeMutation -  implementacja mutacji jedno i dwupunktowej, brzegowej
- `mutationProb` - prawdopodobieństwo mutacji, liczba z zakresu 0,1

#### inversion - implementacja operatora inwersji
- `inversionProb` - prawdopodobieństwo inwersji, liczba z zakresu 0,1

In [1133]:
class Chromosome:
    def __init__(self, bits):
        self.bits = bits
        self.binary = self.setBinary()
        self.decimal = self.setDecimal()
        
    def setBinary(self):
        return [random.randrange(0, 2) for row in range(self.bits)]
    
    def setDecimal(self):
        decimal = 0
        position = len(self.binary)
        for bit in self.binary:
            position -= 1
            decimal += bit * pow(2, position)
        return decimal
    
    def oppositeBit(bit):
        return 1 if bit == 0 else 0
    
    def onePointMutation(self, mutationProb):
        prob = random.random()
        if prob < mutationProb:
            point = random.randrange(0, self.bits)
            self.binary[point] = oppositeBit(self.binary[point])
        self.decimal = self.setDecimal()
        
    def twoPointsMutation(self, mutationProb):
        points = random.sample(range(0, self.bits), 2)
        for point in points:
            prob = random.random()
            if prob < mutationProb:
                self.binary[point] = oppositeBit(self.binary[point])
        self.decimal = self.setDecimal()
        
    def edgeMutation(self, mutationProb):
        points = [0, self.bits - 1]
        for point in points:
            prob = random.random()
            if prob < mutationProb:
                self.binary[point] = oppositeBit(self.binary[point])
        self.decimal = self.setDecimal()
        
    def inversion(self, inversionProb):
        prob = random.random()
        if prob < inversionProb:
            disables = True
            while(disables):
                points = random.sample(range(1, self.bits), 2)
                point1 = min(points)
                point2 = max(points)
                disables = False if (point2 >= point1 + 2) else True
            inversion = self.binary[point1:point2]
            inversion.reverse()
            self.binary = self.binary[:point1] + inversion + self.binary[point2:] 
        self.decimal = self.setDecimal()

#### implementacja konfiguracji wielkości osobnika
- `bits` - liczba bitów chromosomu, liczba całkowita
- `X` - chromosom X osobnika, obiekt typu Chromosome
- `Y` - chromosom Y osobnika, obiekt typu Chromosome

#### boothFunc - implementacja wartości funkcji Booth osobnika

In [1097]:
class Individual:
    def __init__(self, bits):
        self.bits = bits
        self.X = self.setChromosome()
        self.Y = self.setChromosome()
        
    def setChromosome(self):
        return Chromosome(self.bits)
    
    def setX(self, binary):
        self.X.binary = binary
        self.X.decimal = self.X.setDecimal()
        
    def setY(self, binary):
        self.Y.binary = binary
        self.Y.decimal = self.Y.setDecimal()
    
    def boothFunc(self):
        return pow((self.X.decimal + 2 * self.Y.decimal - 7), 2) + pow((2 * self.X.decimal + self.Y.decimal - 5), 2)

#### implementacja populacji
- `chromoBits` - liczba bitów chromosomów osobników populacji, liczba całkowita
- `size` - liczba osobnikow w populacji, liczba całkowita

In [1187]:
class Population:
    def __init__(self, size, chromoBits):
        self.size = size
        self.chromoBits = chromoBits
        if size > 0:
            self.individuals = self.setPopulation()
        else :
            self.individuals = []
        
    def setPopulation(self):    
        return [Individual(self.chromoBits) for row in range(self.size)]
    
    def addIndividual(self, ind):    
        self.individuals.append(ind)
        self.size += 1
        
    def bestSelection(self, percent = 30):
        popAdaptation = []
        bestSelected = []
        N = int(self.size * percent / 100)

        for ind in self.individuals:
            popAdaptation.append(ind.boothFunc())  

        for i in range(0, N):  
            best = max(popAdaptation)
            bestSelected.append(best)
            popAdaptation.remove(best) 

        newPopulation = Population(0, 8)
        for best in bestSelected:
            for ind in self.individuals:
                if ind.boothFunc() == best:
                    newPopulation.addIndividual(ind)
        return newPopulation

In [1190]:
pop = Population(20,8)
newPop = pop.bestSelection(percent = 30)

In [1191]:
newPop.individuals